In [ ]:
# Proyecto ETL – Análisis de Esperanza de Vida y Bienestar Global

In [ ]:
import requests
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Obtener lista de países y territorios del Banco Mundial
resp = requests.get("https://api.worldbank.org/v2/country", params={"format": "json", "per_page": 400})
data = resp.json()
paises = data[1]

# Crear tabla con datos básicos de cada país (DataFrame)
df_countries = pd.DataFrame([
    {
        "name": p["name"],
        "iso3Code": p["id"],
        "region": p["region"]["value"],
        "adminregion": p["adminregion"]["value"]
    }
    for p in paises
])

# Filtro para quitar agregados ( adminregion = "" )
# df_countries = df_countries[df_countries["adminregion"] != ""].reset_index(drop=True)
df_countries = df_countries[df_countries["region"] != "Aggregates"].reset_index(drop=True)

# *************************************
# **********  IMPORTANTE *************
# *************************************
# Comenta y descomenta según convenga para descargar el .csv

descargar = True
# descargar = False


# Usamos función para encapsular toda la lógica de descarga,
# pero ahora descargamos TODOS los países a la vez para cada indicador
# (muchísimo más rápido que hacerlo país por país)
def logica_descarga(indicator_code, start_year=1980, end_year=2023):
    
    # URL para sacar TODA la serie del indicador para TODOS los países
    url = f"https://api.worldbank.org/v2/country/all/indicator/{indicator_code}"

    # Pedimos todo en una sola página (per_page alto)
    params = {
        "format": "json",
        "per_page": 20000,
        "date": f"{start_year}:{end_year}"
    }

    # Hacemos la petición a la API
    resp = requests.get(url, params=params)
    
    # Respuesta en JSON
    data = resp.json()

    # Convertimos la lista a diccionario (iso3, año) -> valor
    # para acceder muy rápido después
    series = {}
    for entry in data[1]:
        iso3 = entry["countryiso3code"]
        year = entry["date"]
        value = entry["value"]
        series[(iso3, year)] = value
    
    return series


if descargar:
    # Los códigos que necesitamos
    esperanza_vida = "SP.DYN.LE00.IN"
    pib = "NY.GDP.PCAP.CD"

    print("Descargando series de todos los países...")

    # >>> Aquí hacemos solo DOS peticiones: una por indicador
    esperanza_series = logica_descarga(esperanza_vida)
    pib_series = logica_descarga(pib)

    print("Descarga completada.")

    # Creamos el CSV donde guardaremos la tabla completa
    with open("data/datos.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["country", "iso3", "region", "year", "life_expectancy", "gdp_per_capita"])
        
        # Recorremos cada país de nuestro DataFrame original
        for i, row in df_countries.iterrows():
            iso3 = row["iso3Code"]
            name = row["name"]
            region = row["region"]

            print(f"Escribiendo datos de {name}")

            # Recorremos cada año y hacemos una fila por año
            for year in range(1980, 2023 + 1):
                y = str(year)

                # Obtenemos los valores ya guardados en los diccionarios
                life = esperanza_series.get((iso3, y))
                gdp = pib_series.get((iso3, y))

                writer.writerow([name, iso3, region, y, life, gdp])

else:
    print("Comprueba, si sale error no la tienes descargada, cambia a descargar = True")


# Cargamos el CSV generado
df = pd.read_csv("data/datos.csv")
df.head()

In [ ]:
df_life = df.copy()

In [ ]:
df_life_media = (
    df_life.groupby(["country", "region"])["life_expectancy"]
    .mean()
    .reset_index()
    .rename(columns={
        "country": "pais",
        "life_expectancy": "esperanza_vida_media"
    })
)

df_life_media.head()

In [ ]:
# 3. Top 10 países con mayor esperanza de vida

top_10 = df_life_media.sort_values("esperanza_vida_media", ascending=False).head(10)
top_10["esperanza_vida_media"] = top_10["esperanza_vida_media"].round(1)

xmin = top_10["esperanza_vida_media"].min() - 0.3
xmax = top_10["esperanza_vida_media"].max() + 0.1

plt.figure(figsize=(8,5))
bars = plt.barh(data=top_10, y="pais", width="esperanza_vida_media")

for bar in bars:
    plt.text(bar.get_width() + 0.02,
             bar.get_y() + bar.get_height()/2,
             f"{bar.get_width():.1f}", va="center")

plt.xlim(xmin, xmax)
plt.xlabel("Esperanza de vida media (años)")
plt.title("Top 10 países con mayor esperanza de vida (1980–2023)")
plt.gca().invert_yaxis()
plt.show()


In [ ]:
# 4. Esperanza de vida media por región

df_final = df_life_media.copy()

region_life_expectancy = (
    df_final.groupby("region")["esperanza_vida_media"]
    .mean()
    .sort_values(ascending=False)
)

xmin = region_life_expectancy.min() - 1
xmax = region_life_expectancy.max() + 1

plt.figure(figsize=(10,6))
region_life_expectancy.plot(kind="bar")
plt.ylim(xmin, xmax)
plt.ylabel("Esperanza de vida media (años)")
plt.title("Esperanza de vida media por región (1980–2023)")
plt.xticks(rotation=45, ha="right")
plt.show()


## 1. Introducción

Este proyecto tiene como objetivo el desarrollo de un proceso **ETL (Extract, Transform, Load)** a partir de datos públicos del **Banco Mundial**, con el fin de analizar la relación entre indicadores económicos y sociales a nivel global.

En concreto, se estudian dos indicadores clave:

- **Esperanza de vida al nacer** (`SP.DYN.LE00.IN`), como medida de bienestar social.
- **PIB per cápita** (`NY.GDP.PCAP.CD`), como medida de desarrollo económico.

El análisis se realiza para el periodo **1980–2023**, garantizando coherencia temporal en todo el proyecto.

---

## 2. Configuración y librerías

In [ ]:
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

---

## 3. ETL – Extracción de datos

Se comprueba si el archivo de datos ya existe localmente. En caso contrario, se descargan los datos desde la API del Banco Mundial.

In [ ]:
DATA_PATH = "data/datos_banco_mundial.csv"
os.makedirs("data", exist_ok=True)

print("¿Existe el archivo de datos?:", os.path.exists(DATA_PATH))

### 3.1 Descarga desde la API (solo si no existe)

In [ ]:
if not os.path.exists(DATA_PATH):
    indicators = {
        "esperanza_vida": "SP.DYN.LE00.IN",
        "pib_per_capita": "NY.GDP.PCAP.CD"
    }

    frames = []

    for nombre, indicador in indicators.items():
        url = f"https://api.worldbank.org/v2/country/all/indicator/{indicador}?format=json&per_page=20000"
        response = requests.get(url)
        data = response.json()[1]
        df_temp = pd.DataFrame(data)
        df_temp["indicador"] = nombre
        frames.append(df_temp)

    df_raw = pd.concat(frames, ignore_index=True)
    df_raw.to_csv(DATA_PATH, index=False)
    print("Datos descargados y guardados correctamente")
else:
    df_raw = pd.read_csv(DATA_PATH)
    print("Datos cargados desde archivo existente")


---

## 4. ETL – Transformación de datos

### 4.1 Limpieza y selección de columnas

In [ ]:
df = df_raw[["country", "date", "value", "indicador"]].copy()

df["pais"] = df["country"].apply(lambda x: eval(x)["value"] if isinstance(x, str) else x)
df["anio"] = pd.to_numeric(df["date"], errors="coerce")
df["valor"] = pd.to_numeric(df["value"], errors="coerce")

### 4.2 Filtrado temporal y eliminación de nulos

In [ ]:
df = df_raw[["country", "date", "value", "indicador"]].copy()

df["pais"] = df["country"].apply(lambda x: eval(x)["value"] if isinstance(x, str) else x)
df["anio"] = pd.to_numeric(df["date"], errors="coerce")
df["valor"] = pd.to_numeric(df["value"], errors="coerce")

### 4.2 Filtrado temporal y eliminación de nulos

In [ ]:
df = df[(df["anio"] >= 1980) & (df["anio"] <= 2023)]
df = df.dropna(subset=["valor"])

### 4.3 Separación de indicadores

In [ ]:
df_life = df[df["indicador"] == "esperanza_vida"]
df_gdp = df[df["indicador"] == "pib_per_capita"]

### 4.4 Media por país (1980–2023)

In [ ]:
df_life_media = (
    df_life.groupby("pais")["valor"]
    .mean()
    .reset_index()
    .rename(columns={"valor": "esperanza_vida_media"})
)

---

## 5. ETL – Carga de datos finales

In [ ]:
df_life_media.to_csv("data/esperanza_vida_media.csv", index=False)
print("Datos transformados guardados correctamente")

---

## 6. Análisis de Datos

## P3. Países con mayor y menor esperanza de vida

### Top 10 países (media 1980–2023)

In [ ]:
top_10 = df_life_media.sort_values("esperanza_vida_media", ascending=False).head(10)

top_10["esperanza_vida_media"] = top_10["esperanza_vida_media"].round(1)

In [ ]:
xmin = top_10["esperanza_vida_media"].min() - 0.3
xmax = top_10["esperanza_vida_media"].max() + 0.1

plt.figure(figsize=(8,5))
bars = plt.barh(top_10["pais"], top_10["esperanza_vida_media"])

for bar in bars:
    plt.text(bar.get_width() + 0.02,
             bar.get_y() + bar.get_height()/2,
             f"{bar.get_width():.1f}", va="center")

plt.xlim(xmin, xmax)
plt.xlabel("Esperanza de vida media (años)")
plt.title("Top 10 países con mayor esperanza de vida (1980–2023)")
plt.gca().invert_yaxis()
plt.show()

---

## P4. Distribución de la esperanza de vida por regiones

### 6.1 Carga de regiones

In [ ]:
url_countries = "https://api.worldbank.org/v2/country?format=json&per_page=500"
countries_data = requests.get(url_countries).json()[1]

df_regions = pd.DataFrame(countries_data)
df_regions = df_regions[["name", "region"]]
df_regions["region"] = df_regions["region"].apply(lambda x: x["value"])

In [ ]:
df_final = df_life_media.merge(df_regions, left_on="pais", right_on="name", how="left")

### 6.2 Media por región

In [ ]:
region_life_expectancy = df_final.groupby("region")["esperanza_vida_media"].mean().sort_values(ascending=False)

In [ ]:
xmin = region_life_expectancy.min() - 1
xmax = region_life_expectancy.max() + 1

In [ ]:
plt.figure(figsize=(10,6))
region_life_expectancy.plot(kind="bar")
plt.ylim(xmin, xmax)
plt.ylabel("Esperanza de vida media (años)")
plt.title("Esperanza de vida media por región (1980–2023)")
plt.xticks(rotation=45, ha="right")
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(data=df_final, x="esperanza_vida_media", y="region")
plt.xlabel("Esperanza de vida media (años)")
plt.title("Distribución de la esperanza de vida por región (1980–2023)")
plt.show()

> Cómo leer el Boxplot (para la exposición)
Te dejo la lectura clara:
Caja corta → región homogénea


> Caja larga → grandes desigualdades internas


> Bigotes largos → países muy distintos entre sí


> Outliers → países que rompen el patrón regional


> 💬 Frase clave:
“Dos regiones pueden tener la misma media, pero realidades internas completamente distintas.”


---

## Distribución regional de la esperanza de vida

A partir de la esperanza de vida media por país en el periodo 1980–2023, se ha analizado su distribución por regiones geográficas. En primer lugar, se calcula la media regional, lo que permite una comparación directa del nivel de bienestar entre regiones.
Para profundizar en el análisis, se utiliza un diagrama de caja que muestra la dispersión interna de cada región. Este enfoque permite identificar regiones con altos niveles de desigualdad interna, así como países que se comportan como valores atípicos respecto a su entorno regional. De este modo, se evidencia que regiones con medias similares pueden presentar realidades sociales muy diferentes.

El análisis de la esperanza de vida media por regiones para el periodo 1980–2023 pone de manifiesto claras diferencias geográficas en los niveles de bienestar. Las regiones con mayor desarrollo socioeconómico presentan, de media, valores más altos de esperanza de vida, lo que refuerza la relación entre crecimiento económico y calidad de vida.
Sin embargo, el uso de diagramas de caja revela que la media regional no refleja por sí sola la realidad completa. Algunas regiones muestran una distribución homogénea, indicando niveles de bienestar relativamente consistentes entre sus países, mientras que otras presentan una elevada dispersión, evidenciando importantes desigualdades internas.
Asimismo, se identifican países que actúan como valores atípicos dentro de su región, lo que sugiere que factores como las políticas públicas, la estabilidad institucional o el acceso a servicios básicos pueden tener un impacto decisivo en la traducción del crecimiento económico en bienestar social. Este análisis refuerza la importancia de complementar los indicadores económicos con variables sociales para obtener una visión más completa del desarrollo global.

---

El análisis muestra una clara relación entre desarrollo económico y esperanza de vida, aunque también revela importantes desigualdades internas entre regiones. La esperanza de vida media permite sintetizar el bienestar a largo plazo y complementa de forma efectiva los indicadores puramente económicos, ofreciendo una visión más humana del desarrollo global.
